<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [1]:
pip install hazm

     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 233 kB 42.6 MB/s 
     |████████████████████████████████| 1.4 MB 33.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=38fd5cd46aee733e9718b2e027966f1e3b64238347012c53361a15b41ad9a137
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154232 sha256=8d63fdfbcb2ad65b07fe444a1a2daeaef81e708df9cc5dbb8111a4e8de3b128a
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


Imports

In [2]:
import re
from os import remove
from hazm import word_tokenize

from tensorflow import keras
from keras.preprocessing import text
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt

Reading txt file and getting verses

Here we need to do some processes

In [5]:
# with this function we check that it contains number in the text or not
def hasNumbers(inputString) :
  for character in inputString :
    if character.isdigit() :
      return True
  return False

def hasDash(inputString) :
  for character in inputString :
    if character == '-' :
      return True
  return False 


with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  dataSet = f.readlines()

verses = []
for line in dataSet :
  if line != ' ' and line != '\n' and hasNumbers(line) == False  and hasDash(line)==False:
    if dataSet.index(line) != 0 :
      verses.append(line.replace('\t',' '))


# for i in range(20):
#   print(verses[i])


In [6]:
with open('/content/stopwords.txt') as f:
  stops = f.readlines()
stopWords = []
for stop in stops :
  stopWords.append(re.sub('\n','',stop.replace('\ufeff','')))
print(stopWords)

['!', '"', '#', '(', ')', '*', ',', '-', '.', '/', ':', '[', ']', '«', '»', '،', '؛', '؟اتفاقا', 'اتفاقا', 'احتراما', 'احتمالا', 'اري', 'اری', 'آری', 'آري', 'از', 'ازجمله', 'اساسا', 'است', 'اش', 'اشكارا', 'آشكارا', 'اصلا', 'اصولا', 'اغلب', 'اكثرا', 'اكنون', 'الان', 'الآن', 'البته', 'ام', 'اما', 'امد', 'آمد', 'امدم', 'آمد', 'امدن', 'آمدن', 'امدند', 'آمدند', 'امده', 'آمده', 'امدي', 'آمدی', 'امديد', 'آمدید', 'امديم', 'آمدیم', 'امروزه', 'امسال', 'امشب', 'ان', 'آن', 'اند', 'انشاالله', 'انصافا', 'انطور', 'آنطور', 'انقدر', 'آنقدر', 'انها', 'آنها', 'انچنان', 'آنچنان', 'انگار', 'او', 'اورد', 'آورد', 'اوردم', 'آوردم', 'اوردن', 'آوردن', 'اوردند', 'آوردند', 'اورده', 'آورده', 'اوردي', 'اوردی', 'آوردی', 'اورديد', 'آوردید', 'اورديم', 'آوردیم', 'اورم', 'آورم', 'اورند', 'آورند', 'اوري', 'اوری', 'آوری', 'اوريد', 'آورید', 'اوريم', 'آوری', 'اولا', 'اي', 'آی', 'ای', 'ايا', 'آیا', 'ايد', 'ايشان', 'ايم', 'اين', 'ايند', 'آيند', 'اينطور', 'اينقدر', 'اينك', 'اینک', 'اينها', 'اينچنين', 'اينگونه', 'ايي', 'ایی', '

Now it's time to use **Hazm** **Library** to tokenize each sentence

In [8]:
concept_words = []
for verse in verses:
  # now we must add all words in the list to words list
  for word in word_tokenize(verse):
    concept_words.append(word)

Remove stop words from verses

In [9]:
# for removing stop words from our words list we need to this process
corpus  = [] #concept words list is a list that contains list of words without stopWords
for word in concept_words :
 if word not in stopWords:
   corpus.append(word)
 
print(corpus)

['گرم', 'سیم', 'درم', 'مرا', 'مونس', 'کم', 'وگر', 'یارم', 'فقیرستی', 'زر', 'فارغ', 'غم', 'خدایا', 'حرمت', 'مردان', 'دنیا', 'فارغش', 'گردان', 'فارغستی', 'کم', 'نگارا', 'مرا', 'خواهی_وگر', 'همدرد', 'همراهی', 'مکن', 'مخور', 'حسرت', 'بختم', 'محتشم', 'بتا', 'زیبا', 'نیکویی', 'رها', 'کن', 'این', 'گدارویی', 'چشم', 'سیرستی', 'فلک', 'حشم', 'طمع', 'آدمی', 'خویش', 'بیگانه', 'وگر', 'طمع', 'خال', 'عم', 'بیا', 'شو', 'مه', 'نعمت', 'جو', 'دولت', 'جو', 'ابلیس', 'این', 'شه', 'صاحب', 'علم', 'ابلیسی', 'سقط', 'ثنا', 'جفا', 'وفا', 'سقم', 'کرم', 'زهی', 'اقبال', 'درویشی', 'زهی', 'اسرار', 'خویشی', 'دانستیی', 'پیشت', 'عدم', 'جهانی', 'هیچان', 'خیال', 'خواب', 'پیچان', 'وگر', 'خفته', 'بدانستی', 'خوابم', 'غم', 'خیالی', 'بیند', 'این', 'خفته', 'اندیشه', 'فرورفته', 'وگر', 'خواب', 'آشفته', 'بجستی', 'نعم', 'زندان', 'غم', 'دیده', 'باغ', 'ارم', 'دیده', 'وگر', 'بیدار', 'گشتی', 'زندان', 'نی', 'ارم', 'امیر', 'دل', 'گوید', 'دلی', 'عاشق', 'گیری', 'نان', 'جامه', 'بیزاری', 'قحط', 'نان', 'کند', 'عشق', 'خبازی', 'وگر', 'گم', 'گشت',



**Tokenize** corpus vocabulary

Import keras and use it 

In [10]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)

wordToIndex = tokenizer.word_index
indexToWord = {v:k for k,v in wordToIndex.items()}
print(wordToIndex)
print(indexToWord[346])

vocab_size = len(wordToIndex) + 1
embed_size = 100

# print(corpus)
windows = [[wordToIndex[w] for w in text.text_to_word_sequence(doc)] for doc in corpus]
print(vocab_size)
print(wordToIndex.items())
# Now each unique word from the corpus is a part of our vocabulary now with a unique numeric identifier



{'این': 1, 'جان': 2, 'دل': 3, 'عشق': 4, 'سر': 5, 'نی': 6, 'کن': 7, 'شمس': 8, 'صد': 9, 'خوش': 10, 'جهان': 11, 'چشم': 12, 'ای': 13, 'دست': 14, 'آب': 15, 'دم': 16, 'آتش': 17, 'مرا': 18, 'کند': 19, 'عالم': 20, 'شب': 21, 'زان': 22, 'مه': 23, 'نیست': 24, 'آید': 25, 'نور': 26, 'عقل': 27, 'گل': 28, 'کردی': 29, 'دیده': 30, 'بس': 31, 'تن': 32, 'روز': 33, 'خاک': 34, 'تویی': 35, 'ماه': 36, 'تبریز': 37, 'غم': 38, 'جمله': 39, 'خانه': 40, 'خویش': 41, 'مست': 42, 'آخر': 43, 'ست': 44, 'آنک': 45, 'کو': 46, 'ره': 47, 'لطف': 48, 'میان': 49, 'پر': 50, 'دیدی': 51, 'جانی': 52, 'همچو': 53, 'شکر': 54, 'عاشق': 55, 'بحر': 56, 'خون': 57, 'باری': 58, 'شاه': 59, 'یار': 60, 'خورشید': 61, 'شه': 62, 'ساقی': 63, 'گشتی': 64, 'تبریزی': 65, 'زهی': 66, 'دانی': 67, 'وگر': 68, 'باده': 69, 'چرخ': 70, 'خدا': 71, 'شو': 72, 'مستی': 73, 'باد': 74, 'گوید': 75, 'دریا': 76, 'رفتی': 77, 'پنهان': 78, 'پرده': 79, 'پای': 80, 'باغ': 81, 'زر': 82, 'روح': 83, 'رخ': 84, 'کرده': 85, 'کار': 86, 'بیا': 87, 'برون': 88, 'عجب': 89, 'هله': 90, 'خمش

Now,we want a list of our sentences as list of words:

In [16]:
sentences = []
for verse in verses:
  sentence = []
  for word in verse.split():
    if word not in stopWords:
      sentence.append(word)
  sentences.append(sentence)
  
print(sentences)

[['گرم', 'سیم', 'درم', 'مرا', 'مونس', 'کم', 'وگر', 'یارم', 'فقیرستی', 'زر', 'فارغ', 'غم'], ['خدایا', 'حرمت', 'مردان', 'دنیا', 'فارغش', 'گردان', 'فارغستی', 'کم'], ['نگارا', 'مرا', 'وگر', 'همدرد', 'همراهی', 'مکن', 'مخور', 'حسرت', 'بختم', 'محتشم'], ['بتا', 'زیبا', 'نیکویی', 'رها', 'کن', 'این', 'گدارویی', 'چشم', 'سیرستی', 'فلک', 'حشم'], ['طمع', 'آدمی', 'خویش', 'بیگانه', 'وگر', 'طمع', 'خال', 'عم'], ['بیا', 'شو', 'مه', 'نعمت', 'جو', 'دولت', 'جو', 'ابلیس', 'این', 'شه', 'صاحب', 'علم'], ['ابلیسی', 'سقط', 'ثنا', 'جفا', 'وفا', 'سقم', 'کرم'], ['زهی', 'اقبال', 'درویشی', 'زهی', 'اسرار', 'خویشی', 'دانستیی', 'پیشت', 'عدم'], ['جهانی', 'هیچان', 'خیال', 'خواب', 'پیچان', 'وگر', 'خفته', 'بدانستی', 'خوابم', 'غم'], ['خیالی', 'بیند', 'این', 'خفته', 'اندیشه', 'فرورفته', 'وگر', 'خواب', 'آشفته', 'بجستی', 'نعم'], ['زندان', 'غم', 'دیده', 'باغ', 'ارم', 'دیده', 'وگر', 'بیدار', 'گشتی', 'زندان', 'نی', 'ارم'], ['امیر', 'دل', 'گوید', 'دلی', 'عاشق', 'گیری', 'نان', 'جامه', 'بیزاری'], ['قحط', 'نان', 'کند', 'عشق', 'خبازی', 

Now we will train our data

In [17]:
data = []
WINDOW_SIZE = 2
for sentence in sentences :
  for word_index , word in enumerate(sentence) :
    for nb_word in sentence[max(word_index - WINDOW_SIZE,0) : min(word_index + WINDOW_SIZE , len(sentences)) + 1] :
      if nb_word != word :
        data.append([word,nb_word])


# print(data[1:20])

[['گرم', 'درم'], ['سیم', 'گرم'], ['سیم', 'درم'], ['سیم', 'مرا'], ['درم', 'گرم'], ['درم', 'سیم'], ['درم', 'مرا'], ['درم', 'مونس'], ['مرا', 'سیم'], ['مرا', 'درم'], ['مرا', 'مونس'], ['مرا', 'کم'], ['مونس', 'درم'], ['مونس', 'مرا'], ['مونس', 'کم'], ['مونس', 'وگر'], ['کم', 'مرا'], ['کم', 'مونس'], ['کم', 'وگر']]


Using Skip-Gram 

In [ ]:
from keras.preprocessing.sequence import skipgrams

# define skip-grams
skip_grams = [skipgrams(window,vocabulary_size=vocab_size,window_size=10) for window in windows]
print(skip_grams)

[([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([[118, 3130], [118, 68], [68, 1350], [68, 118]], [0, 1, 0, 1]), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([],